In [1]:
import cartopy.crs as ccrs
import intake
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import numpy as np
import xarray as xr
from tqdm.autonotebook import tqdm
import pyremo as pr
import cordex as cx
import glob
import regionmask

ERROR 1: PROJ: proj_create_from_database: Open of /work/ch0636/g300096/Python/envs/ikercordex/share/proj failed
/tmp/ipykernel_81290/2325055468.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# create variable info class for model
class modelclass:
    def __init__(self, userexp, runame):
        self.userexp = userexp # XXXYYY
        self.runame = runame # name of the run

In [3]:
# define the runs to be plotted
modelruns =[] # init

modelruns.append(modelclass("056524","REMO2015"))
modelruns.append(modelclass("036030","REMO2020$_{27}$"))
modelruns.append(modelclass("036031","REMO2020$_{27}$ Shallow"))
modelruns.append(modelclass("036032","REMO2020$_{49}$"))
modelruns.append(modelclass("036033","REMO2020$_{49}$ no-Prog"))
modelruns.append(modelclass("036036","REMO2020$_{49}$ Wetcore"))
modelruns.append(modelclass("036037","REMO2020$_{49}$ Shallow"))

In [8]:
# datapath and land-sea-mask (BLA) file
dpath = "./data_in/model_data/histogram/"
remoblafile = "./data_in/model_data/REMO_static/BLA_EUR-11.nc"

In [9]:
# results path
respath = dpath

In [10]:
# define star year and end year
ystart=2001
yend=2010

In [11]:
# open land-sea-mask
mask_lsm = xr.open_dataset(remoblafile).BLA.rename("mask_lsm")

In [12]:
# create prudence mask
mask_pru = cx.regions.prudence.mask_3D(mask_lsm.lon, mask_lsm.lat).rename("mask_pru")

In [13]:
# Set halo zone length (will be cutted out)
halo = 8

In [14]:
# we want to keep attributes when addid dataset
xr.set_options(keep_attrs=True)

In [15]:
# give parameters for the bins
bin_start = 0
bin_end = 140
bin_step = 1

In [16]:
# months for analysis
anmons = np.array([6,7,8])

In [17]:
for obj in modelruns: # here compute things are not really needed, probably
    # due to data size, we need to open year-by-year
    for year in range(ystart,yend+1):
        # load data
        ds =  pr.parse_dates(xr.open_dataset(dpath+"totpre2D_hourly_"+obj.userexp+"_"+str(year)+".nc"))
        # combine data with masks and chooce only summer months
        dsmask = xr.merge([(ds["APRL"]+ds["APRC"]).rename("totpre").where(ds.time.dt.month.isin(anmons), drop=True), mask_pru, mask_lsm], compat="override", join="override")
        # cut halo
        dsmask = dsmask.isel(rlat=slice(halo+1,dsmask.sizes["rlat"]-halo+1), rlon=slice(halo+1,dsmask.sizes["rlon"]-halo+1)).compute()
        # calculate data without sea points
        nosea = dsmask["totpre"].where(dsmask.mask_lsm > 0, drop=True).rename("totpre").compute()
        # loop over regions
        jj = 0
        for reg in dsmask.region:
            hist, bin_edges = np.histogram(nosea.where(dsmask.mask_pru.isel(region=reg),drop=True), bins=np.arange(bin_start,bin_end+bin_step,bin_step))
            xr_tmp = xr.Dataset(
                data_vars=dict(varri=(["bins"],hist,dict(standard_name=str(dsmask.isel(region=reg).names.values)))),
                coords=dict(bins=(["bins"],bin_edges[:-1]+bin_step/2.)),
                attrs=dict({"runname": obj.runame})
            ).rename({"varri": str(dsmask.isel(region=reg).abbrevs.values)}).compute()
            if(jj == 0):
                histstack = xr_tmp
            else:
                histstack = xr.merge([histstack, xr_tmp],compat="override", join="override")
            jj = jj + 1
            del xr_tmp
        if(ystart != yend): # more than one year
            if(year == ystart):
                collect_data = histstack
            else:
                collect_data = collect_data + histstack
        else: # only one year
            collect_data = histstack
        # clear memory
        del ds, dsmask, nosea, histstack
    # write to file
    collect_data.to_netcdf(respath+"prudence_1h_precipitation_hist_"+obj.userexp+"_"+str(ystart)+"-"+str(yend)+".nc",mode="w")
    # clear memory
    del collect_data